In [113]:
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import re
import pandas as pd
import sys
import time
import tweepy
import nltk
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
from TwitterAPI import TwitterAPI

In [114]:
consumer_key = 'fmEX9EvoyQ5UkJgEhn3K0ckuq'
consumer_secret = 'IKcBtkaBKUEWfEtZJC7e1lSgrx8pzsYXpMW8oh570pfGOxi5eb'
access_token = '4602561163-2uZhnk8R6lXlbnEOtdZhkiER5z8S05FEH9QLnve'
access_token_secret = 'nycUV1POzAz9Uy5QOHlqgaXp00BFEcYv5k9cvtKtQDWGN'

In [115]:
def get_twitter():
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)


In [116]:
def gettweet(name):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    outtweets=[]
    tweets = []
    for i in name:
        alltweets = []
        newtweet = api.user_timeline(screen_name = i,count = 200)
        alltweets.extend(newtweet)
        old = alltweets[-1].id-1
        while len(newtweet)>0:
            newtweet = api.user_timeline(screen_name = i,count = 200,max_id = old)
            alltweets.extend(newtweet)
            old = alltweets[-1].id-1
        #outtweets.extend([i,tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"),tweet.retweet_count,tweet.favorite_count,tweet] for tweet in alltweets)
        for tweet in alltweets:
            parse = {}
            parse['name'] = i
            parse['id'] = tweet.id_str
            parse['date'] = tweet.created_at
            parse['text'] = tweet.text.encode("utf-8")
            parse['polarity'] = getsenti(tweet.text)
            parse['retweet'] = tweet.retweet_count
            parse['fav'] = tweet.favorite_count
            outtweets.append(parse)
    #print(outtweets)
    keys = outtweets[0].keys()
    with open('trainer.csv', 'w') as f:
        writer = csv.DictWriter(f,keys)
        #writer.writerow(["name","id","created_at","text","Polarity","Retweet Count","Fav Count"])
        writer.writeheader()
        writer.writerows(outtweets)

In [117]:
def getsenti(tweet):
        analysis = TextBlob(clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return '4'
        elif analysis.sentiment.polarity == 0:
            return '0'
        else:
            return '-4'

In [118]:
def read_screen_names(filename):
    with open(filename) as d:
        names = d.read().splitlines()
    d.close()
    return names

screen_names = read_screen_names('trainer.txt')
print('Read screen names: %s' % screen_names)

Read screen names: ['BarackObama', 'HillaryClinton', 'KingJames', 'serenawilliams', 'katyperry', 'justinbieber', 'jk_rowling', 'johngreen', 'SpaceX', 'pepsi', 'elonmusk', 'mtbarra']


In [119]:
def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", tweet).split())

In [120]:
gettweet(screen_names) 